# k3d usage for discretisedfield

In [1]:
import k3d
import pylab
import matplotlib
import numpy as np
import discretisedfield as df
import matplotlib.pyplot as plt
%matplotlib inline

### A simple vector field to be visualised

Sample is a 5 nm radius sphere sample. The initial field is in the $x$ direction.

In [96]:
r = 3
mesh = df.Mesh(p1=(-r, -r, -r), p2=(r, r, r), cell=(1, 1, 1))

def norm_fun(pos):
    x, y, z = pos
    if x**2 + y**2 + z**2 < r**2:
        return 0.5
    else:
        return 0
    
def val(pos):
    x, y, z = pos
    if x >= 0:
        return (1, 0, 0)
    else:
        return (-1, 0, 0)

field = df.Field(mesh, value=val, norm=norm_fun)

## Vector field

#### Coordinates

Get the spatial coordinates of vectors. These coordinates are centres off cells in a finite difference mesh.

In [97]:
coordinates = np.array(list(field.mesh.coordinates))
coordinates.shape

(216, 3)

Plot cell coordinates

In [98]:
plot = k3d.plot()
plt_coordinates = k3d.points(coordinates)
plot += plt_coordinates
plot.display()

/home/begm/miniconda3/envs/joommf/lib/python3.6/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))


Output()

Reduce the size of points

In [74]:
plt_coordinates.point_size = 0.1

Change the color of points

In [75]:
plt_coordinates.color = 0xff0000

Accordingly, the coordinates function is:

In [76]:
def k3d_coordinates(field, plot=None):
    coordinates = np.array(list(field.mesh.coordinates))
    
    if plot is None:
        plot = k3d.plot()
    plt_coordinates = k3d.points(coordinates)
    plt_coordinates.point_size = 0.1
    plt_coordinates.color = 0xff0000
    plot += plt_coordinates
    if plot is None:
        plot.display()
    
k3d_coordinates(field)

/home/begm/miniconda3/envs/joommf/lib/python3.6/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))


#### Vectors

Get the vectors from the `df.Field` object.

In [77]:
vectors = field.array.copy().reshape(-1, 3)
vectors.shape

(216, 3)

Plot vectors

In [78]:
plot = k3d.plot()
plt_vectors = k3d.vectors(coordinates, vectors)
plot += plt_vectors
plot.display()

/home/begm/miniconda3/envs/joommf/lib/python3.6/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))


Output()

However, the zero vectors are plotted with wrong directions. In order to avoid this problem, we have to plot only those coordinates/vectors where the norm of vector is non-zero.

In [79]:
def select_nonzero(coordinates, vectors):
    nonzero_indices = (np.sum(vectors**2, axis=-1) != 0)
    return coordinates[nonzero_indices], vectors[nonzero_indices]

nonzero_coordinates, nonzero_vectors = select_nonzero(coordinates, vectors)

Now, we can plot only the nonzero vectors.

In [80]:
plot = k3d.plot()
plt_vectors = k3d.vectors(nonzero_coordinates, nonzero_vectors)
plot += plt_vectors
plot.display()

/home/begm/miniconda3/envs/joommf/lib/python3.6/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))


Output()

Now, we need to check the position of vectors relative to the coordinates. This is because we need the middle of a vector to be located at the coordinate.

In [81]:
k3d_coordinates(field, plot=plot)

/home/begm/miniconda3/envs/joommf/lib/python3.6/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))


The tails of vectors are located at coordinates. Therefore, the coordinates for vector field should be moved by 1/2 of their norm.

In [82]:
shifted_nonzero_coordinates = nonzero_coordinates - 0.5 * nonzero_vectors

Now, we plot both vectors with shifted origins and coordinates.

In [83]:
plot = k3d.plot()
plt_vectors = k3d.vectors(shifted_nonzero_coordinates, nonzero_vectors)
plot += plt_vectors
plot.display()

k3d_coordinates(field, plot=plot)

/home/begm/miniconda3/envs/joommf/lib/python3.6/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))


Output()

### Adding colormap

If we want to use different colors for the vectors, we use the color set from the package **pylab** (`pylab.cm.get_cmap()`).

In [84]:
def colormap(name, n=256):
    """Returns a list of tuple (r, g, b) colors."""
    cmap = pylab.cm.get_cmap(name, n)
    cmap_list = []
    for i in range(cmap.N):
        cmap_list.append(cmap(i)[:3])

    return np.array(cmap_list)

colormap('viridis').shape

(256, 3)

We find the minimum and maximum values of a vector component (`component={0, 1, 2}`) and divide the range by 256. If the field is constant (max - min == 0), we use the middle value from the colormap.

In [85]:
component = 0

vector_component = nonzero_vectors[:, component]

def get_int_component(vector_component, avoid_zero=False):
    max_value = vector_component.max()
    min_value = vector_component.min()
    value_range = max_value - min_value

    # Put values in 0-255 range
    if value_range != 0:
        int_component = (vector_component + abs(min_value)) / value_range * 255
    else:
        int_component = [128] * len(vector_component)  # place in the middle of colormap

    return np.vectorize(lambda x: int(x))(int_component)

int_component = get_int_component(vector_component)

Divide values of vectors and colormap set into 256 bins, and match each vector with a specific color.

In [86]:
cmap = colormap('viridis', 256)

colors = cmap[int_component]
colors = ['0x{}'.format(matplotlib.colors.rgb2hex(rgb)[1:]) for rgb in colors]
colors = [(int(i, 16), int(i, 16)) for i in colors]  # tuple (head-color, body-color)

Plotting vector field.

In [92]:
plot = k3d.plot()
plt = k3d.vectors(nonzero_coordinates, nonzero_vectors, colors=colors)
plot += plt
plot.display()

/home/begm/miniconda3/envs/joommf/lib/python3.6/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))


Output()

**Questions**:

1. How to add the colorbar?
2. Can we have cones instead of arrows?
3. What is this warning: `/home/begm/miniconda3/envs/joommf/lib/python3.6/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created. np.dtype(self.dtype).name))`?

# Scalar filed

Create a 10 x 10 x 10 nm sample. The initial field is directed along the X axis, in a volume bounded by a sphere with a diameter of 5 nm.

In [94]:
field_array = field.array.copy()
array_shape = field.array.shape

nx, ny, nz, _ = array_shape

norm = np.linalg.norm(field_array, axis=3)[..., None]

for i in range(nx):
    for j in range(ny):
        for k in range(nz):
            if norm[i, j, k] == 0:
                field_array[i, j, k] = np.nan
            
component = 0

field_component = field_array[..., component]

def get_int_component(field_component):
    max_value = np.nanmax(field_component)
    min_value = np.nanmin(field_component)
    value_range = max_value - min_value

    nx, ny, nz = field_component.shape

    # Put values in 0-255 range
    if value_range != 0:
        int_component = (field_component + abs(min_value)) / value_range * 254
        int_component += 1
    else:
        int_component = 128 * np.ones(field_component.shape)  # place in the middle of colormap

    for i in range(nx):
        for j in range(ny):
            for k in range(nz):
                if np.isnan(field_component[i, j, k]):
                    int_component[i, j, k] = 0
                else:
                    int_component[i, j, k] = int(int_component[i, j, k])
    
    
    
    return int_component

int_component = get_int_component(field_component)

Plotting scalor field. Using existed colormap set.

In [95]:
xmin,ymin,zmin = field.mesh.pmin
xmax,ymax,zmax = field.mesh.pmax

plot = k3d.plot()
plt_vox = k3d.voxels(int_component, color_map=colors,
                               xmin=xmin,xmax=xmax,
                               ymin=ymin,ymax=ymax,
                               zmin=zmin,zmax=zmax, outlines=False)

plot += plt_vox
plot.display()

/home/begm/miniconda3/envs/joommf/lib/python3.6/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "uint8". A coerced copy has been created.
  np.dtype(self.dtype).name))


Output()

**Questions:**
1. How to add the colorbar?
2. Can we change the labels on axes?

# Slice vector field

Visualization of a vector field on a selected plane. Using the function `field.mesh.plane (axis = value)` we get the coordinate values of the origin of the vectors and their magnitude. In the process, we filter those vectors whose length is equal to zero.

We obtain the coordinates of the origin of the vectors and their length on the selected plane.

In [124]:
coordinates_nonzero_slice = []
vectors_nonzero_slice = []
for i in field.mesh.plane(x=2.2):
    vector = field(i)
    norm = vector[0]**2 + vector[1]**2 + vector[2]**2
    if norm > 0:
        coordinates_nonzero_slice.append(i)
        vectors_nonzero_slice.append(vector)
        
coordinates_nonzero_slice = np.array(coordinates_nonzero_slice)
vectors_nonzero_slice = np.array(vectors_nonzero_slice)

shifted_nonzero_coordinates_slice = coordinates_nonzero_slice - 0.5 * vectors_nonzero_slice

Plot

In [125]:
plot = k3d.plot()
vector_plot  = k3d.vectors(shifted_nonzero_coordinates_slice, vectors_nonzero_slice, color=color)

plt_coordinates = k3d.points(coordinates_nonzero_slice)
plt_coordinates.point_size = 0.1
plt_coordinates.color = 0xff0000
plot += plt_coordinates

plot += vector_plot
plot.display()

/home/begm/miniconda3/envs/joommf/lib/python3.6/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))


Output()

# Slice scalor field

Select the field values on the plane

In [ ]:
ith = 1
value_slice = field.array[:,:,ith,1]
nz,ny,nx,_  = field.array.shape

Create a list of triples of node numbers

In [ ]:
ni,nj = value_slice.shape
indices = []
for i in range(ni-1):
    for j in range(nj-1):
        indices.append((i*nj+j, i*nj+j+1, (i+1)*nj+j))        # lower-left triangle
        indices.append(((i+1)*nj+j+1, i*nj+j+1, (i+1)*nj+j))  # upper-right triangle

Select the mesh coordinate

In [ ]:
origins = np.array(list(field.mesh.coordinates))
origins_slice = origins.reshape(nz,ny,nx,3)[:,:,ith,:]

Plot

In [ ]:
plot = k3d.plot()
plt_mesh = k3d.mesh(origins_slice, indices, attribute=value_slice,
                    color_map=k3d.basic_color_maps.CoolWarm,
                    color_range=[-1.0, 1.0])
plot += plt_mesh
plot.display()

# Overlap vector and scalor fields

## 3D

In [ ]:
plot = k3d.plot()
plt_scalor  = k3d.volume(field_scalor.array, bounds=[-size, size, -size, size, -size, size],
                         color_map=np.array(k3d.basic_color_maps.Jet, dtype=np.float32))
plt_vector  = k3d.vectors(origins_nonzero, vector_scale*vectors_nonzero, colors=colors)
plot += plt_scalor
plot += plt_vector
plot.display()

## Slice

In [ ]:
color = 0xff
vector_scale = 1.0

plot = k3d.plot()
plt_vector_slice = k3d.vectors(origins_nonzero_slice, vector_scale*vectors_nonzero_slice, color=color)
plt_scalor_slice = k3d.mesh(origins_slice, indices, attribute=value_slice,
                    color_map=k3d.basic_color_maps.CoolWarm,
                    color_range=[-1.0, 1.0], antialias=True)
plot += plt_mesh
plot += plt
plot.display()

# Mesh as vox

In [87]:
xmin,ymin,zmin = field.mesh.pmin
xmax,ymax,zmax = field.mesh.pmax

plot = k3d.plot()
plt_vox = k3d.voxels(field.array[...,0]!=0, color_map=[0x00ff00],
                               xmin=xmin,xmax=xmax,
                               ymin=ymin,ymax=ymax,
                               zmin=zmin,zmax=zmax)
plot += plt_vox
plot.display()

/home/begm/miniconda3/envs/joommf/lib/python3.6/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "bool" does not match required type "uint8". A coerced copy has been created.
  np.dtype(self.dtype).name))


Output()

In [91]:
myarray = np.ones([2, 1, 2])

xmin,ymin,zmin = 0, 0, 0
xmax,ymax,zmax = myarray.shape

plot = k3d.plot()
plt_vox = k3d.voxels(myarray, color_map=[0x00ff00],
                               xmin=xmin,xmax=xmax,
                               ymin=ymin,ymax=ymax,
                               zmin=zmin,zmax=zmax, outlines=False)
plot += plt_vox
plot.display()

/home/begm/miniconda3/envs/joommf/lib/python3.6/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "uint8". A coerced copy has been created.
  np.dtype(self.dtype).name))


Output()

In [ ]:
myarray

In [ ]:
k3d.voxels?

# Isosurface

Visualization of several isosurfaces.

In [130]:
xmin,ymin,zmin = field.mesh.pmin
xmax,ymax,zmax = field.mesh.pmax

plot = k3d.plot()
plt_iso = k3d.marching_cubes(np.sum(field.array**2,axis=-1), level=0.01,
                             bounds=[xmin,xmax,ymin,ymax,zmin,zmax])

plot += plt_iso
plt_iso = k3d.marching_cubes(np.sum(field.array**2,axis=-1), level=1.0,
                             bounds=[xmin,xmax,ymin,ymax,zmin,zmax],
                             color=0x00ff00)

plot += plt_iso
plot.display()

/home/begm/miniconda3/envs/joommf/lib/python3.6/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))


Output()

**Question**:

* How add alpha chanel for isosurface?
* Is it possible to set the range of values and the step of displaying isosurfaces?
* How add the legend?